In [1]:
import cv2 as cv
import rasterio as rio
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

In [2]:
openFpath = input(r"Open rbnir image file: ")
with rio.open(openFpath) as dst:
    satImg = dst.read()
    satMeta = dst.meta

NameError: name 'maskImg' is not defined

In [13]:
openMaskPath = input(r"Open mask file: ")
with rio.open(openMaskPath) as dst:
    maskImg = dst.read()
    maskMeta = dst.meta

In [5]:
print(satImg.shape)
print(maskImg.shape)

(4, 1073, 1080)
(1, 1073, 1080)


(1073, 1080, 4)

In [14]:
maskImg = maskImg[0,:,:]

In [15]:
maskImg.shape

(1073, 1080)

In [17]:
imgShp = satImg.shape
# zeroImg = np.zeros((imgShp[1], imgShp[2], imgShp[0]))
# zeroImg.shape
# zeroImg = satImg
# zeroImg.shape
zeroImg = np.dstack((satImg[0], satImg[1], satImg[2], satImg[3]))
zeroImg.shape

(1073, 1080, 4)

In [30]:
zeroImg[:,:,0]*= maskImg
zeroImg[:,:,1]*= maskImg
zeroImg[:,:,2]*= maskImg
zeroImg[:,:,3]*= maskImg

In [34]:
satZ = zeroImg.reshape((-1,4))
satZ = np.float32(satZ)

In [46]:
initcriteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 10, 1.0)
deepCriteria = (cv.TERM_CRITERIA_EPS + 3, 10,1.0)

In [37]:
k = 2
see = []
labels = []
knos = []
maxCluster = 30
while k <= maxCluster:
    ret,label,_=cv.kmeans(satZ,k,None,initcriteria,10,cv.KMEANS_RANDOM_CENTERS)
    see.append(ret)
    knos.append(k)
    k+=1

In [43]:
plt.plot(knos, see)
plt.show()

In [54]:
# Deep clustering
k = 10
_,label,_=cv.kmeans(satZ,k,None,deepCriteria,10,cv.KMEANS_RANDOM_CENTERS)


In [55]:
label += 1
labelImg = label.reshape(maskImg.shape)

In [56]:
labelImg.shape

(1073, 1080)

In [57]:
plt.imshow(labelImg)

In [58]:
plt.show()

In [25]:
np.unique(label)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29], dtype=int32)

In [59]:
labelImg = np.uint16(labelImg)

In [60]:
destFolder = input(r"Destination folder of label file: ")
destFpath = Path(destFolder, "cluster_urban_kmeans_10.tif")

In [61]:
satMeta.update(count = 1)

In [63]:
with rio.open(destFpath, 'w', **satMeta) as dst:
    dst.write(labelImg, 1)

In [64]:
canalImg = labelImg
canalImg[canalImg!=10] = 0

In [65]:
destFpath = Path(destFolder, "canalImg.tif")

In [66]:
with rio.open(destFpath, 'w', **satMeta) as dst:
    dst.write(canalImg, 1)